# Estimating wildlife damage to crops using crop insurance data
---

In [1]:
# Clean existing variables
rm(list=ls())
library(plyr)
library(tidyr)
library(dplyr)
library(reshape2)
library(frm)
library(httr)
library(lmtest)
library(sandwich)
library(broom)
library(margins)
library(ggplot2)
library(RColorBrewer)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:plyr’:

    arrange, count, desc, failwith, id, mutate, rename, summarise,
    summarize


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union



Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths


Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric




---
### 1. Download data

TODO: Add some explanation of where the data comes from (url etc.)

In [9]:
getRMA <- function(type, years) {
  # This function downloads individual yearly policy or claim files from the RMA website.  
  # Args:
  #   type: The type of file, must be "policy" or "claim"
  #   years: A vector of integer years. Data from from multiple years will be horizontally stacked
  # Return:
  #   A dataframe
    
  if (type == 'policy') {
      base_url <- "https://www.rma.usda.gov/-/media/RMAweb/SCC-SOB/State-County-Crop-Coverage/sobcov_"
  } else if (type == 'claim') {
      base_url <- "https://www.rma.usda.gov/-/media/RMAweb/Cause-Of-Loss/Summary-of-Business-with-Month-of-Loss/colsom_"
  } else {
      stop("type must equal 'policy' or 'claim'")
  }
    
  # store df from each year in a list
  df_list <- list()
  for (y in 1:length(years)) {
    response <- GET(paste0(base_url, years[y], ".ashx?la=en"))
    writeBin(content(response, as = "raw"), "temp.zip")
    fName <- unzip("temp.zip", list = TRUE)$Name
    unzip("temp.zip")
    df <- read.table(fName, sep = "|", header = FALSE, dec =".", quote = "", 
                     fill=TRUE, skipNul=TRUE, strip.white=TRUE)
    file.remove(c("temp.zip", fName))
    df_list[[y]] <- df  
  }
    
  # vertically concat the list of dataframes
  data <- do.call("rbind", df_list)
    
  return(data)
}


stripWhite <- function(df, cols=c('covcateg', 'quanttype')) {
  # Trim right-side whitespace from strings in specified cols of the df.
  for (col in cols) {
    if (col %in% names(df)) {
        trimws(df[col], which = 'right')
    }
  }
  return(df)   
}


# TODO: The code in this function is will fail silently. Where do these cols come from?
renameCols <- function(df) {
    # Inserts real col names.
    # Args:
    #   df: The dataframe that needs col names
    # Return:
    #   The dataframe with new col names
    
    if (dim(df)[2] == 28) {
      cols <- cols <- c("year", "stfips", "stabb", "cntyfips", "cntyname", "commoditycode", "commodityname", 
                        "insplancode","insplanname", "covcateg", "delivtype", "covlevel", "polsold", "polprem", 
                        "polindemn", "unitssold", "unitsindemn", "quanttype", "acres", "endorsedacres",  
                        "liab", "totalpremium", "subsidies", "indemnityamount", "lossratio")
    }
    if (dim(df)[2] == 30) {
      cols <- cols  <- c("year", "stfips", "stabb", "cntyfips", "cntyname", "commoditycode", "commodityname", 
                         "insplancode","insplanname", "covcateg", "stagecode", "damagecausecode", "damagecausedesc", 
                         "monthloss", "monthname", "polprem", "polindemn",  "acres", "endorsedacres",  
                         "liab", "totalpremium", "subsidies", "lostacres", "indemnityamount", "lossratio")
    }
    names(df) <- cols
    return(df)
}

#### Download policy files

In [ ]:
full_polacres <- getRMA(type='policy', years=c(2017, 2018))
full_polacres <- renameCols(df=full_polacres)
full_polacres <- stripWhite(df=full_polacres)
print(dim(full_polacres))

#### Download claim files

In [ ]:
claims <- getRMA(type='claim', years=c(1990))
claims <- renameCols(df=claims)
claims <- stripWhite(df=claims)
print(dim(claims))

In [5]:
claims

year,stfips,stabb,cntyfips,cntyname,commoditycode,commodityname,insplancode,insplanname,covcateg,⋯,totalpremium,subsidies,lostacres,indemnityamount,lossratio,NA,NA.1,NA.2,NA.3,NA.4
<int>,<int>,<fct>,<int>,<fct>,<int>,<fct>,<int>,<fct>,<fct>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
2017,1,AL,1,Autauga,11,WHEAT,2,RP,A,⋯,9494.000,684.00,137.000,547.000,0,0,0,105.00,3323.00,4.86
2017,1,AL,1,Autauga,11,WHEAT,2,RP,A,⋯,3794.600,739.20,73.800,665.400,0,0,0,39.20,3330.00,4.50
2017,1,AL,1,Autauga,11,WHEAT,2,RP,A,⋯,3320.275,646.80,64.575,582.225,0,0,0,34.30,2913.75,4.50
2017,1,AL,1,Autauga,11,WHEAT,2,RP,A,⋯,1476.200,207.20,20.600,186.600,0,0,0,15.60,1533.60,7.40
2017,1,AL,1,Autauga,11,WHEAT,2,RP,A,⋯,922.625,129.50,12.875,116.625,0,0,0,9.75,958.50,7.40
2017,1,AL,1,Autauga,11,WHEAT,2,RP,A,⋯,2371.625,462.00,46.125,415.875,0,0,0,24.50,2081.25,4.50
2017,1,AL,1,Autauga,11,WHEAT,2,RP,A,⋯,1291.675,181.30,18.025,163.275,0,0,0,13.65,1341.90,7.40
2017,1,AL,1,Autauga,21,COTTON,2,RP,A,⋯,406347.480,46448.82,10681.110,35767.710,0,0,0,732.60,5220.27,0.11
2017,1,AL,1,Autauga,21,COTTON,2,RP,A,⋯,4104.520,469.18,107.890,361.290,0,0,0,7.40,52.73,0.11


---
### 2. Data organization

TODO: add some explanation. There is a figure in the paper we can embed here.

In [43]:
format_policies <- function(df) {
  # TODO: add docstring
  
  df <- reshape(df, idvar = c("year", "stfips", "stabb", "cntyfips", "cntyname", 
                              "commoditycode", "commodityname", "insplancode", 
                              "insplanname", "covcateg", "claim"), 
                timevar = "covlevel", v.names=c("acres", "unitssold"), direction = "wide")
    
  df[is.na(df)] <- 0
    
  # calculate the mean coverage level and the number of units sold by year, stfips, 
  # cntyfips, commoditycode, insplancode, covcateg, claim combination
  df$acres.total <- df$acres.0.95 + df$acres.0.9 + df$acres.0.85 + df$acres.0.8 + df$acres.0.75 + 
                    df$acres.0.7 + df$acres.0.65 + df$acres.0.6 + df$acres.0.55 + df$acres.0.5 
  df$meancov <- (0.95*df$acres.0.95 + 0.9*df$acres.0.9 + 0.85*df$acres.0.85 + 0.8*df$acres.0.8 + 
                 0.75*df$acres.0.75 + 0.7*df$acres.0.7 + 0.65*df$acres.0.65 + 0.6*df$acres.0.6 + 
                 0.55*df$acres.0.55 + 0.5*df$acres.0.5) / df$acres.total
  df$unitssold <- df$unitssold.0.95 + df$unitssold.0.9 + df$unitssold.0.85 + df$unitssold.0.8 + 
                  df$unitssold.0.75 + df$unitssold.0.7 + df$unitssold.0.65 + df$unitssold.0.6 + 
                  df$unitssold.0.55 + df$unitssold.0.5
    
  myvars <- c("year", "stfips", "stabb", "cntyfips", "cntyname", "commoditycode", "commodityname", 
              "insplancode", "insplanname", "covcateg", "claim", "acres.total", "unitssold", "meancov")
  df <- df[myvars]
    
  return(df)
}

In [44]:
# only keep policies with premium expressed in acres and known county fips and commodity code
polacres <- full_polacres[full_polacres$polprem > 0 & full_polacres$cntyfips < 999 & 
                          full_polacres$quanttype == "Acres" & full_polacres$acres > 0 & 
                          full_polacres$commoditycode != 9999, ]
polacres$claim <- ifelse(polacres$unitsindemn > 0, 1, 0)
myvars <- c("year", "stfips", "stabb", "cntyfips", "cntyname", "commoditycode", "commodityname", 
            "insplancode", "insplanname", "covcateg", "covlevel", "unitssold", "claim", "acres")
polacres <- polacres[myvars]
print(dim(polacres))

[1]  0 14


In [36]:
polacres <- format_policies(polacres)

In [37]:
dim(polacres)

[1]  0 14

In [31]:
# we split the policy file into polacresclaim for which we have claims
polacresclaim <- polacres[polacres$claim == 1, ]
dim(polacresclaim)

[1]  0 14

In [28]:
polacresnoclaim <- polacres[polacres$claim == 0, ]
dim(polacresnoclaim)

[1]  0 14

In [27]:
# and polacresnoclaim for which we have no claims. We save this file for later.
polacresnoclaim$lostacres <- 0

ERROR: Error in `$<-.data.frame`(`*tmp*`, lostacres, value = 0): replacement has 1 row, data has 0


In [6]:



# only keep wildlife claims with insured and lost acres and known county fips and commodity code
claimswild <- claims[claims$cntyfips < 999 & claims$acres > 0 & 
                     claims$commoditycode != 9999 & claims$lostacres > 0 & claims$damagecausecode == 93, ]

claimswild<-aggregate(cbind(lostacres) ~ year + stfips + stabb + cntyfips + cntyname + commoditycode + 
                      commodityname + insplancode + insplanname + covcateg, 
                      data = claimswild, FUN = function(x) sum(x), na.action=NULL )
claimspol<-join(polacresclaim, claimswild, by=c("year", "stfips", "stabb", "cntyfips", "cntyname", 
                                               "commoditycode", "commodityname", "insplancode", 
                                               "insplanname", "covcateg"), type='left')

# we can now append the 2 files
claimpoltot <- bind_rows(claimspol, polacresnoclaim)
claimpoltot$lostacres[is.na(claimpoltot$lostacres)] <- 0
claimpoltot$ratio <- claimpoltot$lostacres/claimpoltot$acres.total
claimpoltot$ratio[claimpoltot$ratio > 1] <- 1
claimpoltot$lostacres <- claimpoltot$ratio*claimpoltot$acres
claimpoltot$fullfips <- claimpoltot$stfips*1000 + claimpoltot$cntyfips
claimpoltot$logacres <- log(claimpoltot$acres.total)
claimpoltot$logunitssold <- log(claimpoltot$unitssold)
claimpoltot$cat <- ifelse(claimpoltot$covcateg=="C", 1, 0)
claimpoltot$y2017 <- ifelse(claimpoltot$year==2017, 1, 0)

# create string variables to be treated as categorical variables
claimpoltot$fullfips.ch <- as.character(claimpoltot$fullfips)
claimpoltot$commoditycode.ch <- as.character(claimpoltot$commoditycode)
claimpoltot$insplancode.ch <- as.character(claimpoltot$insplancode)

ERROR: Error in `$<-.data.frame`(`*tmp*`, lostacres, value = 0): replacement has 1 row, data has 0


---
### 3. Modeling

In [ ]:
model_glm = glm(
  ratio ~  meancov + logacres + logunitssold + cat + y2017 + insplancode.ch + commoditycode.ch + fullfips.ch,
  epsilon = 1e-6,
  data = claimpoltot,
  family = quasibinomial
)

# get robust standard errors
tidy(coeftest(model_glm, vcov = vcovHC(model_glm, type="HC")))

summary(model_glm)


In [ ]:
(m <- margins(model_glm))
summary(m)

---
### 4. Results

In [ ]:
# Estimated lost acres if no low coverage
preds <- claimpoltot[claimpoltot$year == 2018,]
preds$meancov <- 1  # 100%
preds$insplancode.ch <- "1"
preds$cat <- 0
preds$ratio.hat <- predict(model_glm, new, se.fit = FALSE, type = "response")

summary(preds$ratio.hat)

preds$lostacres.hat <- preds$acres.total * preds$ratio.hat
preds <- aggregate(cbind(acres.total, lostacres, lostacres.hat) ~ year + fullfips + stfips + stabb + cntyfips + 
                                                                cntyname + commoditycode + commodityname, 
                 data = new, FUN = function(x) sum(x), na.action=NULL )

preds$ratio.county <-preds$lostacres / preds$acres.total
preds$ratio.county.hat <- preds$lostacres.hat / preds$acres.total
summary(preds$ratio.county)
summary(preds$ratio.county.hat)

# write.csv(new, file = "/Users/sophiemckee/Dropbox/APHIS/Insurance/Paper on Wildlife Claims/predictions_ratio_allfips_1.csv",row.names=FALSE)

In [ ]:
# load data 
cty_shape <- read.csv("county_shape.csv")
st_shape <- read.csv("state_shape.csv")
#preds <- read.csv('predictions_ratio_allfips.csv', stringsAsFactors = FALSE)

# organize loss data
cty_shape$Fips = as.character(cty_shape$Fips)
preds$cntyfips <- as.character(preds$cntyfips)
preds$stfips <- as.character(preds$stfips)
preds$fips <- NA

for (i in 1:nrow(preds)) {
  if (nchar(preds$cntyfips[i]) == 1) {
    preds$cntyfips[i] = paste('00', preds$cntyfips[i], sep='')
  }
  if (nchar(preds$cntyfips[i]) == 2) {
    preds$cntyfips[i] = paste('0', preds$cntyfips[i], sep='')
  }
  preds$fips[i] = paste(preds$stfips[i], preds$cntyfips[i], sep='')
}

cty_shape['lost_corn'] <- NA
cty_shape['lost_wheat'] <- NA
cty_shape['lost_soybeans'] <- NA
cty_shape['lost_peanuts'] <- NA

for (i in 1:nrow(preds)) {
  if (preds$commoditycode[i] == 41) {
    cty_shape[cty_shape$Fips == preds$fips[i], 'lost_corn'] = preds$ratiohat[i]
  }
  if (preds$commoditycode[i] == 11) {
    cty_shape[cty_shape$Fips == preds$fips[i], 'lost_wheat'] = preds$ratiohat[i]
  }
  if (preds$commoditycode[i] == 81) {
    cty_shape[cty_shape$Fips == preds$fips[i], 'lost_soybeans'] = preds$ratiohat[i]
  }
  if (preds$commoditycode[i] == 75) {
    cty_shape[cty_shape$Fips == preds$fips[i], 'lost_peanuts'] = preds$ratiohat[i]
  }
}


# map it
ggplot() + 
  theme_void() +
  theme(plot.title = element_text(hjust = 0.5)) +
  ggtitle('Fraction of Peanut Acres Lost to Wildlife') + 
  scale_fill_gradientn('', colours = brewer.pal(9, 'YlOrRd')) +
  # county polygons 
  geom_polygon(data = cty_shape[ order(cty_shape$order), ],
               aes(long, lat, group = group, fill = lost_peanuts),
               col = "#c0c5ce", size=0.1) +
  # cords
  coord_map("bonne",  param = 40) + 
  # state polygons
  geom_polygon(data = st_shape[ order(st_shape$state_order), ],
               aes(state_long, state_lat, group = state_group),
               fill = NA,
               col = "black") 